# A5: GAMSclasses module notebook

# Introduction

This notebook describes the functionality of the GAMSclasses module in the jQMM library (JBEI Quantitative Metabolic Modeling). This module interacts with the GAMS (General Algebraic Modeling System) to solve the necessary optimization problems.

# Setup

First, we need to set the path and environment variable properly:

In [1]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [2]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

Let's import the module we want to test plus other dependencies:

In [3]:
import GAMSclasses as GC
import numpy

And move to a defined directory:

In [4]:
cd ~/tests

/mnt/nfs.jbei/nfs/users/hgmartin/tests


# Classes description

## GAMSSet class

The GAMSSet class is used to store information on sets for GAMS problems (e.g. see http://www.gams.com/help/index.jsp?topic=%2Fgams.doc%2Fuserguides%2Fuserguide%2F_u_g__set_definition.html).

Let's first create an empty set (met set in this case):

In [5]:
testSet = GC.GAMSSet('metset',set([]))  

And then add elements for the set:

In [6]:
lines = ["/","'gluL'","'gly'","'ileL'","'lacL'","'mlthf'","'oaabm'","'pepbm'","'pheL'",
"'proL'","'pyrbm'","'r5pbm'","'serL'","'tyrL'","'valL'","/"]

testSet.read(('metset.txt','\n'.join(lines)))

The final set looks like this:

In [7]:
output= testSet.write('toString')
print output

/ 
'gluL'
'gly'
'ileL'
'lacL'
'mlthf'
'oaabm'
'pepbm'
'pheL'
'proL'
'pyrbm'
'r5pbm'
'serL'
'tyrL'
'valL'
/ 



## GAMSParameter class

The GAMSParameter class stores information on paramters for GAMS problems (https://www.gams.com/help/index.jsp?topic=%2Fgams.doc%2Fuserguides%2Fmccarl%2Fparameters_1.htm).

Let's proceed as for sets and produce first an empty parameter to which we add some data:


In [8]:
testPar = GC.GAMSParameter('S',[])

#testpar.read('/users/hgmartin/pythontest/gams/stoich.txt')

Here is the data for a stochiometry matrix, which are described as parameters in GAMS:

In [9]:
lines = ["S('13dpg','GAPD')=1.0;","S('13dpg','PGK')=-1.0;","S('2pg','ENO_b')=1.0;","S('2pg','ENO_f')=-1.0;",
               "S('2pg','PGM_b')=-1.0;","S('2pg','PGM_f')=1.0;","S('3pgc','PGK')=1.0;"]

In [10]:
testPar.read(('stoich.txt','\n'.join(lines)))

And the GAMS parameter file looks like:

In [11]:
print  testPar.write('toString')


S('13dpg','GAPD')=1.0;
S('13dpg','PGK')=-1.0;
S('2pg','ENO_b')=1.0;
S('2pg','ENO_f')=-1.0;
S('2pg','PGM_b')=-1.0;
S('2pg','PGM_f')=1.0;
S('3pgc','PGK')=1.0;



## GAMSTable and GAMSTableKeys classes

GAMSTable and GAMSTableKeys are classes used to handle GAMS tables:

In [12]:
name  = "testTable"
xkeys = GC.GAMSTableKeys('m',['1','2'])
ykeys = GC.GAMSTableKeys('frag',['Ala','Ser'])
array = numpy.array([[0.2,0.3],[0.4,0.9]])
    
table = GC.GAMSTable(name,xkeys,ykeys,array)

In [13]:
print table.write('toString')

Table testTable(frag,m)
	1	2
Ala	0.20000	0.30000
Ser	0.40000	0.90000
;


##  GAMSProblem

GAMSProblem is used to hold all the information regarding an algebraic problem defined in GAMS. It is also equipped with the necessary methods to solve it.

First let's check that GAMS is callable (status = 0 means a successful call):

In [14]:
status = os.system('gams')
print status

0


The lines for the classic transport problem are stored in the module:

In [15]:
testProblem = GC.getTransportProblem()

In [16]:
for line in testProblem.writeGAMSFile('toString')[1].split('\n'):
    print line

SETS
I   canning plants   / SEATTLE, SAN-DIEGO /
J   markets          / NEW-YORK, CHICAGO, TOPEKA / ;
PARAMETERS
A(I)  capacity of plant i in cases
         /    SEATTLE     350
              SAN-DIEGO   600  /
B(J)  demand at market j in cases
         /    NEW-YORK    325
              CHICAGO     300
              TOPEKA      275  / ;
TABLE D(I,J)  distance in thousands of miles
                    NEW-YORK       CHICAGO      TOPEKA
      SEATTLE          2.5           1.7          1.8   
      SAN-DIEGO        2.5           1.8          1.4  ;
SCALAR F  freight in dollars per case per thousand miles  /90/ ;
PARAMETER C(I,J)  transport cost in thousands of dollars per case ;
       C(I,J) = F * D(I,J) / 1000 ;
VARIABLES
       X(I,J)  shipment quantities in cases
       Z       total transportation costs in thousands of dollars ;
POSITIVE VARIABLE X ;
EQUATIONS
       COST        define objective function
       SUPPLY(I)   observe supply limit at plant i
       DEMAND(J)   satisfy 

We can use the included methods to find the solution to the algebraic problem through GAMS:

In [17]:
with testProblem as prob:
    # Run problem
    prob.run()

    # Check if it is done
    prob.waitTilDone()

    # Collect data
    prob.collect()
        
    # Output data 
    resultsDict = prob.Results  
       

And the result is the expected 153.68 (https://www.gams.com/docs/example.htm): 

In [18]:
print resultsDict

{'Zout': 'Z    =       153.68\n'}


## GAMSProblemBatch

In a similar way, it is possible to define batches of GAMS problems than can be solved in serial or parallel:

In [19]:
# Get GAMS problems        
testProblem1 = GC.getTransportProblem('transport1','dir1')
testProblem2 = GC.getTransportProblem('transport2','dir2')

testBatch = GC.GAMSProblemBatch([testProblem1,testProblem2])
testBatch.erase = True
       
       
# Solve testBatch    
with testBatch as batch:
    # Run problem(s)
    batch.run()

    # Check if they are done
    batch.waitTilDone()
        
    # Collect data
    batch.collect()
        
    # Put results in local format 
    resultsDict = batch.getAllResults()
       

In [20]:
print resultsDict['transport1']['Zout']

Z    =       153.68



In [21]:
print resultsDict['transport2']['Zout']

Z    =       153.68

